In [1]:
import sys
sys.path.append('/Users/diegodavila/miniconda3/lib/python3.7/site-packages')

In [31]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import argparse
import glob
import os

In [54]:
def score(sub, path):

    root = ET.parse('/Users/diegodavila/Box/BBL_Related/GRMPY_n_back_scoring/grympytemplate.xml').getroot() #read template xxml file
    scorelabel=root.getchildren()[5].getchildren() #the stimuli scores is in index 5

    bb=pd.read_csv('{}'.format(path),skiprows=6,sep='\t',header=None) #read logfile for a particular subjects
    bb.columns=['Subject','Trial','EventType','Code','Time','TTime','Uncertainty0','Duration','Uncertainty1',
           'ReqTime','ReqDur','StimType','PairIndex']

    back0=[] #0back
    back1=[] # 1back
    back2=[] #2back
    for i in range(0,len(scorelabel)):
        if scorelabel[i].get('category') == '0BACK':
            back0.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])
        elif scorelabel[i].get('category') == '1BACK':
            back1.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])
        elif scorelabel[i].get('category') == '2BACK':
            back2.append([scorelabel[i].get('expected'),scorelabel[i].get('index')])

    # each list consists of both results with NR means No result and Macth means correct result as it is on xml
    # how to comppute final score? maye be (number of  Match/( number of NR + number of Match))!!
    # you combine all the output in one file may be  in json
    scoresummary={'0BACK':back0,'1BACK':back1,'2BACK':back2}
    c=list(scoresummary.items())

    allback=[]
    templateback0=c[0][1]
    templateback1=c[1][1]
    templateback2=c[2][1]

    for i in range(0,len(templateback0)):
        a1=bb[bb['Trial'] == np.int(templateback0[i][1])]
        aa=a1['TTime'].to_list()
        if len(aa) > 2 :
            response=aa[1]/10
        else :
            response=0
        allback.append([c[0][0],templateback0[i][1],templateback0[i][0],response])

    for i in range(0,len(templateback1)):
        a1=bb[bb['Trial'] == np.int(templateback1[i][1])]
        aa=a1['TTime'].to_list()
        if len(aa) > 2 :
            response=aa[1]/10
        else :
            response=0
        allback.append([c[1][0],templateback1[i][1],templateback1[i][0],response])

    for i in range(0,len(templateback2)):
        a1=bb[bb['Trial'] == np.int(templateback2[i][1])]
        aa=a1['TTime'].to_list()
        if len(aa) > 2 :
            response=aa[1]/10
        else :
            response=0
        allback.append([c[2][0],templateback2[i][1],templateback2[i][0],response])

    # output
    dfallback=pd.DataFrame(allback)
    dfallback.columns=['task','Index','Results','ResponseTime']

    zeroback = dfallback[dfallback.task=="0BACK"]
    oneback = dfallback[dfallback.task=="1BACK"]
    twoback = dfallback[dfallback.task=="2BACK"]

    #2back
    twoback_accuracy = []
    twoback_speed_fp = []
    twoback_speed_tp = []


    for i in range(len(twoback["task"])):

        if (twoback.iloc[i,2] == 'NR') & (twoback.iloc[i,3] == 0.0):
            # this is a true negative
            twoback_accuracy.append("TN")
        if (twoback.iloc[i,2] == 'NR') & (twoback.iloc[i,3] != 0.0):
            # this is a false positive
            twoback_accuracy.append("FP")
            twoback_speed_fp.append(twoback.iloc[i,3])
        if (twoback.iloc[i,2] == 'Match') & (twoback.iloc[i,3] == 0.0):
            #this is a false negative
            twoback_accuracy.append("FN")
        if (twoback.iloc[i,2] == 'Match') & (twoback.iloc[i,3] != 0.0):
            # this is a true positive
            twoback_accuracy.append("TP")
            twoback_speed_tp.append(twoback.iloc[i,3])
    twoback_speed_fp = np.mean(twoback_speed_fp)
    twoback_speed_tp = np.mean(twoback_speed_tp)

    #oneback
    oneback_accuracy = []
    oneback_speed_fp = []
    oneback_speed_tp = []
    for i in range(len(oneback["task"])):

        if (oneback.iloc[i,2] == 'NR') & (oneback.iloc[i,3] == 0.0):
            # this is a true negative
            oneback_accuracy.append("TN")
        if (oneback.iloc[i,2] == 'NR') & (oneback.iloc[i,3] != 0.0):
            # this is a false positive
            oneback_accuracy.append("FP")
            oneback_speed_fp.append(oneback.iloc[i,3])
        if (oneback.iloc[i,2] == 'Match') & (oneback.iloc[i,3] == 0.0):
            #this is a false negative
            oneback_accuracy.append("FN")
        if (oneback.iloc[i,2] == 'Match') & (oneback.iloc[i,3] != 0.0):
            # this is a true positive
            oneback_accuracy.append("TP")
            oneback_speed_tp.append(oneback.iloc[i,3])
    oneback_speed_fp = np.mean(oneback_speed_fp)
    oneback_speed_tp = np.mean(oneback_speed_tp)

    #zero back
    zeroback_accuracy = []
    zeroback_speed_fp = []
    zeroback_speed_tp = []
    for i in range(len(zeroback["task"])):

        if (zeroback.iloc[i,2] == 'NR') & (zeroback.iloc[i,3] == 0.0):
            # this is a true negative
            zeroback_accuracy.append("TN")
        if (zeroback.iloc[i,2] == 'NR') & (zeroback.iloc[i,3] != 0.0):
            # this is a false positive
            zeroback_accuracy.append("FP")
            zeroback_speed_fp.append(zeroback.iloc[i,3])
        if (zeroback.iloc[i,2] == 'Match') & (zeroback.iloc[i,3] == 0.0):
            #this is a false negative
            zeroback_accuracy.append("FN")
        if (zeroback.iloc[i,2] == 'Match') & (zeroback.iloc[i,3] != 0.0):
            # this is a true positive
            zeroback_accuracy.append("TP")
            zeroback_speed_tp.append(zeroback.iloc[i,3])
    zeroback_speed_fp = np.mean(zeroback_speed_fp)
    zeroback_speed_tp = np.mean(zeroback_speed_tp)

    twobackNumFP = twoback_accuracy.count("FP")
    twobackNumTP = twoback_accuracy.count("TP")
    twobackNumFN = twoback_accuracy.count("FN")
    twobackNumTN = twoback_accuracy.count("TN")

    onebackNumFP = oneback_accuracy.count("FP")
    onebackNumTP = oneback_accuracy.count("TP")
    onebackNumFN = oneback_accuracy.count("FN")
    onebackNumTN = oneback_accuracy.count("TN")

    zerobackNumFP = zeroback_accuracy.count("FP")
    zerobackNumTP = zeroback_accuracy.count("TP")
    zerobackNumFN = zeroback_accuracy.count("FN")
    zerobackNumTN = zeroback_accuracy.count("TN")

    from scipy.stats import norm
    import math
    Z = norm.ppf

    def dp(hits, misses, fas, crs):
        #
        # hits = True Positive
        # misses = False Negative
        # fas = False Positive
        # crs = True Negative
        #
        # Floors an ceilings are replaced by half hits and half FA's
        half_hit = 0.5 / (hits + misses)
        half_fa = 0.5 / (fas + crs)

        # Calculate hit_rate and avoid d' infinity
        hit_rate = hits / (hits + misses)
        if hit_rate == 1:
            hit_rate = 1 - half_hit
        if hit_rate == 0:
            hit_rate = half_hit

        # Calculate false alarm rate and avoid d' infinity
        fa_rate = fas / (fas + crs)
        if fa_rate == 1:
            fa_rate = 1 - half_fa
        if fa_rate == 0:
            fa_rate = half_fa

        # Return d'
        dprime = Z(hit_rate) - Z(fa_rate)
        return(dprime)


    twoback_dPrime = dp(twobackNumTP,twobackNumFN,twobackNumFP,twobackNumTN)
    oneback_dPrime = dp(onebackNumTP,onebackNumFN,onebackNumFP,onebackNumTN)
    zeroback_dPrime = dp(zerobackNumTP,zerobackNumFN,zerobackNumFP,zerobackNumTN)

    data = {
        "twobackNumFN": twobackNumFN,
        "twobackNumFP": twobackNumFP,
        "twobackNumTN": twobackNumTN,
        "twobackNumTP": twobackNumTP,

        "onebackNumFN": onebackNumFN,
        "onebackNumFP": onebackNumFP,
        "onebackNumTN": onebackNumTN,
        "onebackNumTP": onebackNumTP,

        "zerobackNumFN": zerobackNumFN,
        "zerobackNumFP": zerobackNumFP,
        "zerobackNumTN": zerobackNumTN,
        "zerobackNumTP": zerobackNumTP,


        "twoback_speed_fp": twoback_speed_fp,
        "twoback_speed_tp": twoback_speed_tp,

        "oneback_speed_fp": oneback_speed_fp,
        "oneback_speed_tp": oneback_speed_tp,

        "zeroback_speed_fp": zeroback_speed_fp,
        "zeroback_speed_tp": zeroback_speed_tp,

        "twoback_dPrime": twoback_dPrime,
        "oneback_dPrime": oneback_dPrime,
        "zeroback_dPrime": zeroback_dPrime
    }

    output = pd.DataFrame(data, columns =
        ["twobackNumFN",
        "twobackNumFP",
        "twobackNumTN",
        "twobackNumTP",

        "onebackNumFN",
        "onebackNumFP",
        "onebackNumTN",
        "onebackNumTP",

        "zerobackNumFN",
        "zerobackNumFP",
        "zerobackNumTN",
        "zerobackNumTP",

        "twoback_speed_fp",
        "twoback_speed_tp",

        "oneback_speed_fp",
        "oneback_speed_tp",

        "zeroback_speed_fp",
        "zeroback_speed_tp",

        "twoback_dPrime",
        "oneback_dPrime",
        "zeroback_dPrime"], index = ["{}".format(sub)])
    output.index.name = 'bblid'
    
    return output

In [21]:
dirList = glob.glob("/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/*/*/*/*-frac2B_1.00.log")

In [22]:
dirList

['/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/102041/10603/18/102041-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/103679/10110/13/103679-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/104235/10585/18/104235-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/104785/10779/18/104785-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/105168/10761/18/105168-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/105176/10571/18/105176-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/105767/10708/18/105767-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scitran/bbl/GRMPY_822831/105860/10641/19/105860-frac2B_1.00.log',
 '/Users/diegodavila/Box/BBL_Related/GRMPY_LOGFILES/scit

In [9]:
import flywheel
fw = flywheel.Client()
proj = fw.lookup('bbl/GRMPY_822831')
sessions = proj.sessions()
sessions = [fw.get(x.id) for x in sessions]

In [38]:
def is_empty(file_name):
    """ Check if file is empty by reading first character in it"""
    # open ile in read mode
    with open(file_name, 'r') as read_obj:
        # read first character
        one_char = read_obj.read(1)
        # if not fetched then file is empty
        if not one_char:
            return True
    return False

In [58]:
o = pd.DataFrame()
for s in sessions:
    for d in dirList:
        if (s.subject.label in d):
            try:
                newrow = score(s.subject.label,d)
                o = pd.concat([newrow, o], ignore_index=False)
            except: 
                print('There was a problem with {}'.format(s.subject.label))
o

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  after removing the cwd from sys.path.
/Users/diegodavila/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/diegodavila/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


There was a problem with 20247
There was a problem with 116354
There was a problem with 90077


,twobackNumFN,twobackNumFP,twobackNumTN,twobackNumTP,onebackNumFN,onebackNumFP,onebackNumTN,onebackNumTP,zerobackNumFN,zerobackNumFP,...,zerobackNumTP,twoback_speed_fp,twoback_speed_tp,oneback_speed_fp,oneback_speed_tp,zeroback_speed_fp,zeroback_speed_tp,twoback_dPrime,oneback_dPrime,zeroback_dPrime
bblid,,,,,,,,,,,,,,,,,,,,,
21085,0,3,42,15,0,1,44,15,0,1,...,15,533.200000,426.160000,598.600000,471.893333,362.000000,431.580000,3.335001,3.843789,3.843789
21085,2,4,41,13,0,0,45,15,0,1,...,15,575.700000,630.400000,NaN,453.766667,512.800000,475.946667,2.458400,4.120463,3.843789
21056,7,7,38,8,3,3,42,12,1,2,...,14,1041.385714,925.937500,1131.200000,703.391667,1041.800000,565.857143,1.096545,2.342707,3.202374
21016,1,1,44,14,0,1,44,15,1,1,...,14,449.500000,457.942857,465.700000,478.873333,394.000000,494.678571,3.510961,3.843789,3.510961
20949,1,5,40,14,0,1,44,15,0,1,...,15,642.160000,488.328571,312.300000,379.580000,340.700000,356.300000,2.721726,3.843789,3.843789
20974,9,1,44,6,6,0,45,9,0,0,...,15,676.500000,468.516667,NaN,540.744444,NaN,501.560000,1.756528,2.539895,4.120463
20963,0,2,43,15,0,0,45,15,0,0,...,15,684.700000,643.513333,NaN,541.020000,NaN,473.520000,3.535203,4.120463,4.120463
20941,4,5,40,11,0,0,45,15,0,2,...,15,458.460000,394.172727,NaN,452.993333,398.400000,448.766667,1.843566,4.120463,3.535203
20940,14,7,38,1,15,2,43,0,0,1,...,15,826.957143,562.600000,1174.300000,NaN,756.800000,758.700000,-0.488193,-0.132626,3.843789


In [59]:
o.to_csv("/Users/diegodavila/Box/BBL_Related/GRMPY_n_back_scoring/GRMPY_fullnBackScore.csv")